In [ ]:
import random
import time
import numpy as np

In [ ]:
def quick_pivot_first(arr):
    if len(arr) == 0:
        return []
    pivot = arr[0]
    left = []
    right = []
    
    left = list(filter(lambda x: x < pivot, arr))
    right = list(filter(lambda x: x > pivot, arr))
    
    return quick_pivot_first(left) + [pivot] + quick_pivot_first(right)

def partition(arr, left, right):
    pivot = arr[right]
    index = left
    for i in range(left, right):
        if arr[i] <= pivot:
            arr[i], arr[index] = arr[index], arr[i]
            index += 1
    arr[index], arr[right] = arr[right], arr[index]
    return index

def partition_random(arr, left, right):
    n = random.randint(left, right)
    arr[n], arr[right] = arr[right], arr[n]
    return partition(arr, left, right)

def quick_random(arr, low, high):
    if low >= high:
        return []
    left = []
    right = []

    index = random.randint(low, high)
    pivot = arr[index]
    left = list(filter(lambda x: x < pivot, arr))
    right = list(filter(lambda x: x > pivot, arr))
    return quick_random(left, 0, len(left)-1) + [pivot] + quick_random(right, 0, len(right)-1)


n = 100
arr1 = random.sample(range(n), n)
arr2 = arr1.copy()
len_ = len(arr2) - 1

In [ ]:
start = time.time()
quick_pivot_first(arr1)
print(str(n)+" elements, pivot(first), runtime: "+str(time.time() - start))

start = time.time()
a = quick_random(arr2, 0, len_)
print(str(n)+" elements, pivot(random), runtime: "+str(time.time() - start))

# PySpark

In [ ]:
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()

n = 1000

data = random.sample(range(n), n)
data = range(n)
rdd = sc.parallelize(data)

def quicksort(rdd, th):
    n = rdd.count()
    if n <= th:  
        return quick_random(rdd.collect(), 0, n-1)
    
    pivot = rdd.takeSample(False, 1)
    pivot = pivot[0] 
    left = rdd.filter(lambda x: x < pivot)
    right = rdd.filter(lambda x: x >= pivot)
    
    return quicksort(left, th) + quicksort(right, th)

start = time.time()
result = quicksort(rdd, max(rdd.count() * 0.3, 1000))
end = time.time()
print(str(end - start))

with open("runtime.txt", "a") as f:
    f.write(str(n) +"\n")
    f.write(str(end - start) + "\n")